## Test environment

In [1]:
import primer3
import pandas as pd

# Import modules
from multiplexdesigner.utils import MultiplexPanel

In [2]:
# Convert different sections to DataFrames
def convert_primer_data_to_tables(data):
    tables = {}
    
    # 1. Primer Pairs Summary
    if 'PRIMER_PAIR' in data:
        primer_pairs_df = pd.DataFrame(data['PRIMER_PAIR'])
        primer_pairs_df.index.name = 'Pair_ID'
        primer_pairs_df = primer_pairs_df.round(3)  # Round decimals
        tables['primer_pairs'] = primer_pairs_df
    
    # 2. Left Primers
    if 'PRIMER_LEFT' in data:
        left_primers_df = pd.DataFrame(data['PRIMER_LEFT'])
        left_primers_df.index.name = 'Left_Primer_ID'
        left_primers_df = left_primers_df.round(3)
        tables['left_primers'] = left_primers_df
    
    # 3. Right Primers  
    if 'PRIMER_RIGHT' in data:
        right_primers_df = pd.DataFrame(data['PRIMER_RIGHT'])
        right_primers_df.index.name = 'Right_Primer_ID'
        right_primers_df = right_primers_df.round(3)
        tables['right_primers'] = right_primers_df
    
    # 4. Internal Primers
    if 'PRIMER_INTERNAL' in data:
        internal_primers_df = pd.DataFrame(data['PRIMER_INTERNAL'])
        internal_primers_df.index.name = 'Internal_Primer_ID'
        internal_primers_df = internal_primers_df.round(3)
        tables['internal_primers'] = internal_primers_df
    
    # 5. Summary Statistics
    summary_data = {
        'Metric': ['Left Primers Returned', 'Right Primers Returned', 'Internal Primers Returned', 'Primer Pairs Returned'],
        'Count': [
            data.get('PRIMER_LEFT_NUM_RETURNED', 0),
            data.get('PRIMER_RIGHT_NUM_RETURNED', 0), 
            data.get('PRIMER_INTERNAL_NUM_RETURNED', 0),
            data.get('PRIMER_PAIR_NUM_RETURNED', 0)
        ]
    }
    tables['summary'] = pd.DataFrame(summary_data)
    
    return tables

def create_primer_dataframe(primer_data):
    """
    Convert primer design results to organized pandas DataFrames
    """
    
    # Method 1: Create a comprehensive primer pairs DataFrame
    primer_pairs = []
    
    num_pairs = primer_data.get('PRIMER_PAIR_NUM_RETURNED', 0)
    
    for i in range(num_pairs):
        pair_info = {
            'pair_id': i,
            'pair_penalty': primer_data.get(f'PRIMER_PAIR_{i}_PENALTY'),
            'product_size': primer_data.get(f'PRIMER_PAIR_{i}_PRODUCT_SIZE'),
            'product_tm': primer_data.get(f'PRIMER_PAIR_{i}_PRODUCT_TM'),
            'compl_any_th': primer_data.get(f'PRIMER_PAIR_{i}_COMPL_ANY_TH'),
            'compl_end_th': primer_data.get(f'PRIMER_PAIR_{i}_COMPL_END_TH'),
            
            # Left primer info
            'left_sequence': primer_data.get(f'PRIMER_LEFT_{i}_SEQUENCE'),
            'left_coords': primer_data.get(f'PRIMER_LEFT_{i}'),
            'left_tm': primer_data.get(f'PRIMER_LEFT_{i}_TM'),
            'left_gc_percent': primer_data.get(f'PRIMER_LEFT_{i}_GC_PERCENT'),
            'left_penalty': primer_data.get(f'PRIMER_LEFT_{i}_PENALTY'),
            'left_self_any_th': primer_data.get(f'PRIMER_LEFT_{i}_SELF_ANY_TH'),
            'left_self_end_th': primer_data.get(f'PRIMER_LEFT_{i}_SELF_END_TH'),
            'left_hairpin_th': primer_data.get(f'PRIMER_LEFT_{i}_HAIRPIN_TH'),
            'left_end_stability': primer_data.get(f'PRIMER_LEFT_{i}_END_STABILITY'),
            
            # Right primer info
            'right_sequence': primer_data.get(f'PRIMER_RIGHT_{i}_SEQUENCE'),
            'right_coords': primer_data.get(f'PRIMER_RIGHT_{i}'),
            'right_tm': primer_data.get(f'PRIMER_RIGHT_{i}_TM'),
            'right_gc_percent': primer_data.get(f'PRIMER_RIGHT_{i}_GC_PERCENT'),
            'right_penalty': primer_data.get(f'PRIMER_RIGHT_{i}_PENALTY'),
            'right_self_any_th': primer_data.get(f'PRIMER_RIGHT_{i}_SELF_ANY_TH'),
            'right_self_end_th': primer_data.get(f'PRIMER_RIGHT_{i}_SELF_END_TH'),
            'right_hairpin_th': primer_data.get(f'PRIMER_RIGHT_{i}_HAIRPIN_TH'),
            'right_end_stability': primer_data.get(f'PRIMER_RIGHT_{i}_END_STABILITY'),
        }
        primer_pairs.append(pair_info)
    
    df_pairs = pd.DataFrame(primer_pairs)

    return(df_pairs)


In [3]:
# Get the mutation list (junctions) around which to design primer pairs
panel = MultiplexPanel("test_panel", "hg38")
panel.load_config(config_path="./config/designer_default_config.json")
panel.import_junctions_csv("./data/design_regions.csv")

Successfully imported 4 junctions from ./data/design_regions.csv


In [4]:
print(panel.junctions)

[Junction(EGFR_T790M, chr7:55181378-55181378), Junction(KRAS_G12D, chr12:25245350-25245350), Junction(KRAS_G13R, chr12:25245348-25245349), Junction(BRAF_V600E, chr7:140753336-140753336)]


In [5]:
panel.merge_close_junctions()

Merging junctions within 100 bp on same chromosome...
Merged 2 junctions: KRAS_G13R_KRAS_G12D
After merging: 3 junctions remain


In [6]:
print(panel.junctions)

[Junction(KRAS_G13R_KRAS_G12D, chr12:25245348-25245350), Junction(EGFR_T790M, chr7:55181378-55181378), Junction(BRAF_V600E, chr7:140753336-140753336)]


In [7]:
fasta_file = '/Users/ctosimsen/hg38/hg38.fa'
panel.extract_design_regions_from_fasta(fasta_file, padding=200)

Extracting design regions from /Users/ctosimsen/hg38/hg38.fa with 200bp padding...
Error extracting region for KRAS_G13R_KRAS_G12D: [Errno 2] No such file or directory: '/Users/ctosimsen/hg38/hg38.fa'
Error extracting region for EGFR_T790M: [Errno 2] No such file or directory: '/Users/ctosimsen/hg38/hg38.fa'
Error extracting region for BRAF_V600E: [Errno 2] No such file or directory: '/Users/ctosimsen/hg38/hg38.fa'
Successfully extracted 0 design regions


In [8]:
for junction in panel.junctions:
    if hasattr(junction, 'design_region'):
        print(f"\n{junction.name}:")
        print(f"  Region: {junction.design_region} ")



KRAS_G13R_KRAS_G12D:
  Region: None 

EGFR_T790M:
  Region: None 

BRAF_V600E:
  Region: None 


In [9]:
print(vars(panel.junctions[2]))

{'name': 'BRAF_V600E', 'chrom': 'chr7', 'five_prime': 140753336, 'three_prime': 140753336, 'design_region': None, 'design_start': None, 'design_end': None, 'junction_length': None, 'jmin_coordinate': None, 'jmax_coordinate': None}


In [10]:
panel.calculate_junction_coordinates_in_design_region()

Calculated junction coordinates for 0 junctions


In [11]:
print(vars(panel.junctions[2]))

{'name': 'BRAF_V600E', 'chrom': 'chr7', 'five_prime': 140753336, 'three_prime': 140753336, 'design_region': None, 'design_start': None, 'design_end': None, 'junction_length': None, 'jmin_coordinate': None, 'jmax_coordinate': None}


In [12]:
panel.design_config

{'opt_melting_temperature': 60,
 'primer_pair_max_tm_difference': 5.0,
 'tm_threshold': 3.0,
 'primer_optimal_length': 22,
 'primer_length_range': [15, 30],
 'amplicon_gap_range': [20, 60],
 'max_amplicon_length': 100,
 'junction_padding_bases': 3,
 '5_primer_tail': 'GGACACTCTTTCCCTACACGACGCTCTTCCGATCTAAAAAAAAAAAAAAAAAAAATGGGAAAGAGTGTCC',
 '3_prime_tail': 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT',
 'primer_length_penalty': 1.0,
 'primer_complexity_penalty': 1.0,
 'amplicon_length_penalty': 1.0,
 'primer_min_gc': 30,
 'primer_max_gc': 70,
 'primer_gc_clamp': 0,
 'max_poly_x': 5,
 'PRIMER_MAX_SELF_ANY_TH': 45.0,
 'PRIMER_MAX_SELF_END_TH': 35.0,
 'PRIMER_PAIR_MAX_COMPL_ANY_TH': 45.0,
 'PRIMER_PAIR_MAX_COMPL_END_TH': 35.0,
 'PRIMER_MAX_HAIRPIN_TH': 24.0,
 'PRIMER_MAX_TEMPLATE_MISPRIMING_TH': 40.0,
 'PRIMER_PAIR_MAX_TEMPLATE_MISPRIMING_TH': 70.0}

In [13]:
panel.pcr_config

{'annealing_temperature': 60,
 'primer_concentration': 50,
 'dntp_concentration': 0.6,
 'dna_concentration': 50,
 'mv_concentration': 50,
 'dv_concentration': 1.5,
 'dmso_concentration': 0.0,
 'dmso_fact': 0.6,
 'formamide_concentration': 0.8,
 'salt_correction_formula': 'santalucia1998',
 'thermodynamic_table': 'santalucia1998'}

In [14]:
min_product_length = 2 * panel.design_config['primer_length_range'][0] + junction.jmax_coordinate - junction.jmin_coordinate
max_product_length = panel.design_config['max_amplicon_length']


# Set global design arguments
global_args={
    'PRIMER_TM_FORMULA': 1,
    'PRIMER_SALT_CORRECTIONS': 1,
    'PRIMER_OPT_SIZE': panel.design_config['primer_optimal_length'],
    'PRIMER_MIN_SIZE': panel.design_config['primer_length_range'][0],
    'PRIMER_MAX_SIZE': panel.design_config['primer_length_range'][1],
    'PRIMER_PRODUCT_OPT_SIZE': 60,
    'PRIMER_PAIR_WT_PRODUCT_SIZE_LT': 0.5,
    'PRIMER_PAIR_WT_PRODUCT_SIZE_GT': 2.0,
    'PRIMER_OPT_TM': panel.design_config['opt_melting_temperature'],
    'PRIMER_MIN_TM': panel.design_config['opt_melting_temperature']-panel.design_config['tm_threshold'],
    'PRIMER_MAX_TM': panel.design_config['opt_melting_temperature']+panel.design_config['tm_threshold'],
    'PRIMER_SALT_MONOVALENT': panel.pcr_config['mv_concentration'],           # The millimolar (mM) concentration of monovalent salt cations (usually KCl) in the PCR.
    'PRIMER_SALT_DIVALENT': panel.pcr_config['dv_concentration'],             # The millimolar concentration of divalent salt cations (usually MgCl^(2+)) in the PCR. 
    'PRIMER_DNA_CONC': panel.pcr_config['primer_concentration'],
    'PRIMER_DNTP_CONC': panel.pcr_config['dntp_concentration'],               # Millimolar concentration of the sum of all deoxyribonucleotide triphosphates (e.g. 4x 0.2=0.8)
    'PRIMER_DMSO_CONC': panel.pcr_config['dmso_concentration'],
    'PRIMER_FORMAMIDE_CONC': panel.pcr_config['formamide_concentration'],
    'PRIMER_NUM_RETURN': 10,                                                   # The maximum number of primer (pairs) to return.
    'PRIMER_OPT_GC_PERCENT': 50.0,                                             # Optimum GC percent. This parameter influences primer selection only if PRIMER_WT_GC_PERCENT_GT or PRIMER_WT_GC_PERCENT_LT are non-0.
    'PRIMER_MIN_GC': panel.design_config['primer_min_gc'],
    'PRIMER_MAX_GC': panel.design_config['primer_max_gc'],
    'PRIMER_MAX_END_GC': 4,
    'PRIMER_MAX_END_STABILITY': 9,
    'PRIMER_MAX_POLY_X': panel.design_config['max_poly_x'],
    'PRIMER_MAX_NS_ACCEPTED': 0,
    'PRIMER_PAIR_MAX_DIFF_TM': panel.design_config['primer_pair_max_tm_difference'],
    'PRIMER_MAX_SELF_ANY': 8.0,
    'PRIMER_MAX_SELF_END': 3.0,
    'PRIMER_PAIR_MAX_COMPL_ANY': 8.0,
    'PRIMER_PAIR_MAX_COMPL_END': 8.0,
    'PRIMER_MAX_SELF_ANY_TH': 45.0,
    'PRIMER_MAX_SELF_END_TH': 35.0,
    'PRIMER_PAIR_MAX_COMPL_ANY_TH': 45.0,
    'PRIMER_PAIR_MAX_COMPL_END_TH': 35.0,
    'PRIMER_MAX_HAIRPIN_TH': 24.0,
    'PRIMER_MAX_TEMPLATE_MISPRIMING_TH': 40.0,
    'PRIMER_PAIR_MAX_TEMPLATE_MISPRIMING_TH': 70.0,
    'PRIMER_WT_SIZE_LT': panel.design_config['primer_length_penalty'],       # Penalty weight for primers shorter than PRIMER_OPT_SIZE.
    'PRIMER_WT_SIZE_GT': panel.design_config['primer_length_penalty'],       # Penalty weight for primers longer than PRIMER_OPT_SIZE.
    'PRIMER_WT_GC_PERCENT_LT': 0.0,
    'PRIMER_WT_GC_PERCENT_GT': 0.0,
    'PRIMER_WT_SELF_ANY': 5.0,
    'PRIMER_WT_SELF_ANY_TH': 5.0,
    'PRIMER_WT_HAIRPIN_TH': 1.0,
    'PRIMER_WT_TEMPLATE_MISPRIMING_TH': 1.0,
    'PRIMER_PAIR_WT_COMPL_ANY': 5.0,
    'PRIMER_PAIR_WT_COMPL_ANY_TH': 5.0,
    'PRIMER_PAIR_WT_COMPL_END_TH': 5.0,
    'PRIMER_PAIR_WT_TEMPLATE_MISPRIMING_TH': 1.0,
    'PRIMER_THERMODYNAMIC_OLIGO_ALIGNMENT': 0,                               # If set to 1, use thermodynamic values (parameters ending in "_TH")
    'PRIMER_THERMODYNAMIC_TEMPLATE_ALIGNMENT': 0,
    'PRIMER_PRODUCT_SIZE_RANGE': [[min_product_length, max_product_length]]
}


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [ ]:
results_list = []


for junction in panel.junctions:
    # Set the sequence arguments
    seq_args={
        'SEQUENCE_ID': junction.name,
        'SEQUENCE_TEMPLATE': junction.design_region,
        'SEQUENCE_PRIMER_PAIR_OK_REGION_LIST': [[1,junction.jmin_coordinate,junction.jmax_coordinate,junction.junction_length-junction.jmax_coordinate]]
    }

    #print(global_args)

    results = primer3.bindings.design_primers(
        seq_args=seq_args,
        global_args=global_args
    )

    # Generate tables
    tables = convert_primer_data_to_tables(results)

    results_list.append(results)

    # Display tables
    #for table_name, df in tables.items():
    #    print(f"\n=== {table_name.upper().replace('_', ' ')} ===")
    #    print(df.to_string())
    #    print()

    # Save to Excel with multiple sheets
    with pd.ExcelWriter('.out/' + junction.name + '_primer_analysis.xlsx', engine='openpyxl') as writer:
        for sheet_name, df in tables.items():
            df.to_excel(writer, sheet_name=sheet_name, index=True)

    # Save individual CSV files
    for table_name, df in tables.items():
        df.to_csv(f'.out/{junction.name}_primer_{table_name}.csv', index=True)

    print("Tables saved to:")
    print("- primer_analysis.xlsx (multi-sheet Excel file)")
    print("- Individual CSV files for each table")

In [ ]:
print(seq_args)

In [ ]:
print(results_list[1])

In [ ]:
display(create_primer_dataframe(results_list[2]))

In [ ]:
oligo_calc = primer3.thermoanalysis.ThermoAnalysis()

oligo_calc.set_thermo_args(
    mv_conc = 50.0,
    dv_conc = 1.5,
    dntp_conc = 0.6,
    dna_conc = 50.0,
    dmso_conc = 0.0,
    dmso_fact = 0.6,
    formamide_conc = 0.0,
    annealing_temp_c = -10,
    temp_c = 37.0,
    max_nn_length = 60,
    max_loop = 30,
    tm_method = 'santalucia',
    salt_corrections_method = 'santalucia'
)

oligo_calc.calc_hairpin('TGCTTGCTCTGATAGGAAAATGA')

In [ ]:
primer_list = ['TGATGGGACCCACTCCATCG','ACAGTAAAAATAGGTGATTTTGGTCTAGC', 'CTGATGGGACCCACTCCATCG', 'ACAGTAAAAATAGGTGATTTTGGTCTAGC']

In [ ]:
for primer in primer_list:
    print(oligo_calc.calc_tm(primer))  # Print the melting temp

In [ ]:
from itertools import combinations

for primer in primer_list:
    print(oligo_calc.calc_tm(primer))  # Print the melting temp
    print(oligo_calc.calc_homodimer(primer))
    print(oligo_calc.calc_hairpin(primer))

for a, b in combinations(primer_list, 2):
    print(f"Primer 1: {a} + Primer 2: {b}")
    print(oligo_calc.calc_heterodimer(a, b))
    print(oligo_calc.calc_end_stability(a, b))

